# Revenue Management Simple Two-Class Model
## Littlewood's Rule

Reference: https://link.springer.com/article/10.1057/palgrave.rpm.5170134

The Littlewood's rule assumes two product classes (e.g.,full- and discount-fare airline tickets, full- and discount-rate for hotel rooms, etc.) with associated prices $p_f$ and $p_d$, where $p_f > p_d$. The capacity is $C$. We ignore calcellations, no-shows, and overbooking. $D_f$ and $D_d$ denote demands for full-fare and discount-fare, respectively. The demand is a random variable with distribution $F_f$ and $F_d$.

The problem we are trying to solve is to determine how much discount class demand to accept before seeing the realization of full class demand. Why is this the case? In general hotel, airline bookings exihibit similar patterns: demands for discount class arrive first in the booking horizon and demands for the full class arrive later. This is because of two very different customer segments: leisure travelers and business travelers. Their characteristics determine such an interesting arrival pattern. Leisure travelers are price sensitive and tend to book well in advance whereas business travelers are relatively less price sensitive (afterall they are not paying for the ticket or room) and tend to book much closer to the service date. 

With those characteristics, the problem for the company is to determine how much discount class demand to accept before realizing full class demand. What happens if they don't evalaute it carefully? Let's imagine two scenarios:
- Booking limit for the discount class is too low. In this situation, more rooms reserved for late-arriving full class business customers. But if the full class demand is not strong, then the airplane will take off with more empty seats. They could have sold it to the discount class early on when the booking requests arrive. So this causes __spoilage__. Once the airplane take off, the empty seats are spoiled because airlines cannot store it and save for later days.
- Booking limie for the discount class is too high. In htis situation, less rooms are reserved for the late-arriving high paying business class. If the business demand is strong, since they sell too many rooms upfront for the discount class, they forgo the higher revenue they could have earned if they had enough seats for business class. This causes dilution because airlines earn lower price instead of higher price.

The challenge is then to balance the tradeoff with the goal of maximizing expected revenue.

Then, question is how we do it? Well, we do marginal analysis. Suppose that we have $x$ units of capacity remaining and we receive a booking request for a discount class. Two possible outcomes depending on what we decide to do:
- If we accept it, we collect __guaranteed__ revenue $p_d$.
- If we do not accept it, we are hoping that we could sell it to the higher paying class. But it is not guaranteed. We could sell this unit $x$ (marginal $x^{th}$ unit) if and only if demand for full class is at least $x$; if the demand for the full class is lower than $x$, we got nothing from it - spoilage of the seat. 

So the question is what is the expected revenue we could collect from this $x^{th}$ unit? The answer depends on the probability of full class demand being greater than or equal to $x$

$$EV(\text{reserving } x^{th} \text{ seat}) = p_f Pr(D_f \geq x)$$

Now I can make statements as follows:
- We will accept the discount class booking request if $p_d \geq p_f\Pr(D_f \geq x)$
- We will not accept the discount class booking request if $p_d < p_f\Pr(D_f \geq x)$

## Algorithm for Computing Optimal Discount Booking Limits
The marginal analysis we just did suggests that we can use a simple iterative algorithm to calculate the optimal booking limit $b^*$ for discount class.

In [1]:
%matplotlib inline

In [2]:
# Importing the necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy import stats
from scipy.stats import norm

In [3]:
# Setting initial parameter values
# Capacity
cap = 146
# Prices for full-fare and discount-fare
full_price, disc_price = 174, 114
# Mean and standard deviation for full-fare and discount-fare demand
mu_full, sig_full = 92, 30
mu_disc, sig_disc = 80, 25

In [4]:
# Function to calculate the revenue change for a given booking limit
def ev_b_to_b_plus_one(bl):
    # Expected value of moving from the current booking limit to the next one
    ev_b_to_b_plus_one = disc_price - full_price * (1 - norm.cdf(cap - bl, mu_full, sig_full))
    return ev_b_to_b_plus_one

In [5]:
# Algorithm to find the optimal booking limit
b = 0
b_opt = 0

try:
    # If the initial booking limit is equal to the capacity, then the optimal booking limit is the initial booking limit
    if b == cap:
        b_opt = b
    else:    
        # If the expected value of moving from the current booking limit to the next one is less than or equal to 0,
        # or the cumulative probability of the next booking limit is equal to 1,
        # then the optimal booking limit is the current booking limit
        if (ev_b_to_b_plus_one(b) <= 0) or (norm.cdf(b + 1, mu_disc, sig_disc) == 1):
            b_opt = b
        else:
            # Otherwise, keep increasing the booking limit until the expected value of moving to the next booking limit becomes less than or equal to 0
            # or the cumulative probability of the next booking limit becomes equal to 1
            while (ev_b_to_b_plus_one(b) > 0) and (norm.cdf(b + 1, mu_disc, sig_disc) < 1):
                b += 1
            b_opt = b
except Exception as e:
    print("An error occurred while calculating the optimal booking limit.")
    print(f"Error details: {e}")

print(b_opt)

66


## Verification

Littlewood's rule says that the optimal protection level $y^*$ for the full-fare class such that
$$
\Pr(D_f>y^*) = \dfrac{p_d}{p_f} \quad \Longrightarrow \quad 1-\Pr(D_f \leq y^*) = \dfrac{p_d}{p_f} \quad \Longrightarrow \quad \Pr(D_f \leq y^*) = 1 - \dfrac{p_d}{p_f}
$$
Therefore, we can calculate this optimal protection level $y^*$ by taking the inverse of a normal distribution where the percentile is $1-\dfrac{p_d}{p_f}$:
$$
y^* = F^{-1}(1-\dfrac{p_d}{p_f})
$$

In [6]:
try:
    # Calculate the critical fractile for the protection level of the full-fare class
    cf = 1 - disc_price / full_price
    # Calculate the optimal protection level for the full-fare class using the inverse of the normal distribution
    y_star = norm.ppf(cf, mu_full, sig_full)
    # Calculate the optimal booking limit for the discount-fare class
    bl_opt = cap - y_star
except Exception as e:
    print("An error occurred while calculating the optimal booking limit using Littlewood's rule.")
    print(f"Error details: {e}")

bl_opt

65.97969202048654

In [7]:
# Compare the optimal booking limit derived from the algorithm with the one calculated by Littlewood's rule
b_opt, bl_opt

(66, 65.97969202048654)


## Output Interpretation

The output of the calculations is the optimal booking limit for the discount-fare class, which is determined by both the algorithm and Littlewood's rule. 

The optimal booking limit represents the maximum amount of discount-fare demand that the company should accept before realizing the full-fare demand. By setting the booking limit to this value, the company can balance the trade-off between potential spoilage and dilution, thereby maximizing its expected revenue.

The fact that the optimal booking limit derived from the algorithm is the same as the one calculated by Littlewood's rule serves as a validation of both the algorithm and Littlewood's rule. This shows that Littlewood's rule, despite its simplicity, can provide an effective solution to the problem of setting booking limits in a revenue management context.
